# Marketing Analytics Dashboard
This notebook combines ads, invoice, and order data for comprehensive marketing performance analysis.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set style
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

## Generate Combined Sample Data

In [ ]:
# Generate comprehensive sample data
np.random.seed(42)

# Date range for 2018
months = pd.date_range('2018-01-01', '2018-12-31', freq='MS')

# Marketing channels
mediums = ['SEM', 'Social Media', 'Display Ads', 'Email', 'Events', 
           'Print', 'PR', 'Retargeting', 'Newsletter', 'Webinar', 'Organic']

# Create combined monthly data
combined_data = []

for month in months:
    for medium in mediums:
        # Base metrics with some variation
        base_spend = np.random.uniform(2000, 15000)
        seasonality = 1 + 0.3 * np.sin(month.month * np.pi / 6)  # Seasonal variation
        
        combined_data.append({
            'Month': month,
            'Medium': medium,
            'Ads Spend': base_spend * seasonality * np.random.uniform(0.9, 1.1),
            'Marketing Invoice': base_spend * seasonality * np.random.uniform(0.85, 1.15),
            'Marketing Estimate': base_spend * seasonality,
            'Impressions': np.random.randint(10000, 200000) * seasonality,
            'Clicks': np.random.randint(500, 10000) * seasonality,
            'Total Conversions': np.random.randint(20, 300) * seasonality,
            'Add To Cart': np.random.randint(10, 100) * seasonality,
            'Lead Submission': np.random.randint(15, 150) * seasonality,
            'Whitepaper Download': np.random.randint(5, 50) * seasonality,
            'Total Orders': np.random.randint(5, 80),
            'Revenue': np.random.uniform(5000, 150000) * seasonality,
            'Order Net Amount': np.random.uniform(4500, 140000) * seasonality,
            'Workdays To Fulfill': np.random.uniform(3, 12)
        })

df = pd.DataFrame(combined_data)

# Calculate derived metrics
df['CTR'] = (df['Clicks'] / df['Impressions'] * 100).round(2)
df['CPC'] = (df['Ads Spend'] / df['Clicks']).round(2)
df['Conversion Rate'] = (df['Total Conversions'] / df['Clicks'] * 100).round(2)
df['Cost Per Conversion'] = (df['Ads Spend'] / df['Total Conversions']).round(2)
df['ROAS'] = (df['Revenue'] / df['Ads Spend']).round(2)
df['ROI'] = ((df['Revenue'] - df['Ads Spend']) / df['Ads Spend'] * 100).round(2)

# Add time-based features
df['Year'] = df['Month'].dt.year
df['Quarter'] = df['Month'].dt.quarter
df['Month_No'] = df['Month'].dt.month
df['Month_Name'] = df['Month'].dt.strftime('%B')
df['Month_Year'] = df['Month'].dt.strftime('%b %Y')

# Round currency values
currency_cols = ['Ads Spend', 'Marketing Invoice', 'Marketing Estimate', 
                 'Revenue', 'Order Net Amount', 'Cost Per Conversion']
for col in currency_cols:
    df[col] = df[col].round(2)

# Round integer metrics
integer_cols = ['Impressions', 'Clicks', 'Total Conversions', 
                'Add To Cart', 'Lead Submission', 'Whitepaper Download']
for col in integer_cols:
    df[col] = df[col].round(0).astype(int)

print(f"Generated {len(df)} records across {len(mediums)} mediums and {len(months)} months")
print(f"\nData shape: {df.shape}")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

## Data Summary Statistics

In [ ]:
# Overall summary statistics
print("="*80)
print("MARKETING PERFORMANCE SUMMARY - 2018")
print("="*80)

print(f"\nTotal Marketing Spend: ${df['Ads Spend'].sum():,.2f}")
print(f"Total Marketing Invoice: ${df['Marketing Invoice'].sum():,.2f}")
print(f"Total Revenue: ${df['Revenue'].sum():,.2f}")
print(f"Total Orders: {df['Total Orders'].sum():,}")
print(f"Total Conversions: {df['Total Conversions'].sum():,}")
print(f"Total Impressions: {df['Impressions'].sum():,}")
print(f"Total Clicks: {df['Clicks'].sum():,}")

print(f"\nAverage ROAS: {df['ROAS'].mean():.2f}x")
print(f"Average ROI: {df['ROI'].mean():.2f}%")
print(f"Average CTR: {df['CTR'].mean():.2f}%")
print(f"Average Conversion Rate: {df['Conversion Rate'].mean():.2f}%")
print(f"Average CPC: ${df['CPC'].mean():.2f}")
print(f"Average Cost Per Conversion: ${df['Cost Per Conversion'].mean():.2f}")

## Performance by Medium

In [ ]:
# Aggregate by medium
by_medium = df.groupby('Medium').agg({
    'Ads Spend': 'sum',
    'Revenue': 'sum',
    'Total Orders': 'sum',
    'Total Conversions': 'sum',
    'Impressions': 'sum',
    'Clicks': 'sum',
    'ROAS': 'mean',
    'ROI': 'mean',
    'CTR': 'mean',
    'Conversion Rate': 'mean'
}).round(2)

by_medium = by_medium.sort_values('Ads Spend', ascending=False)

print("\nPerformance by Medium:")
print(by_medium)

## Executive Summary Dashboard

In [ ]:
# Create comprehensive executive dashboard
fig = plt.figure(figsize=(18, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Spend by Medium
ax1 = fig.add_subplot(gs[0, :])
by_medium_sorted = by_medium.sort_values('Ads Spend', ascending=False)
colors = sns.color_palette('husl', len(by_medium_sorted))
bars = ax1.barh(by_medium_sorted.index, by_medium_sorted['Ads Spend'], color=colors)
ax1.set_xlabel('Total Spend ($)', fontsize=12)
ax1.set_title('Marketing Spend by Medium - 2018', fontsize=14, fontweight='bold')
ax1.xaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax1.grid(True, alpha=0.3, axis='x')

# 2. Monthly Spend Trend
ax2 = fig.add_subplot(gs[1, 0])
monthly_spend = df.groupby('Month')['Ads Spend'].sum()
ax2.plot(monthly_spend.index, monthly_spend.values, marker='o', 
         linewidth=2, markersize=6, color='steelblue')
ax2.fill_between(monthly_spend.index, monthly_spend.values, alpha=0.3, color='steelblue')
ax2.set_title('Monthly Spend Trend', fontsize=12, fontweight='bold')
ax2.set_xlabel('Month', fontsize=10)
ax2.set_ylabel('Spend ($)', fontsize=10)
ax2.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax2.tick_params(axis='x', rotation=45, labelsize=8)
ax2.grid(True, alpha=0.3)

# 3. Monthly Revenue Trend
ax3 = fig.add_subplot(gs[1, 1])
monthly_revenue = df.groupby('Month')['Revenue'].sum()
ax3.plot(monthly_revenue.index, monthly_revenue.values, marker='s', 
         linewidth=2, markersize=6, color='darkgreen')
ax3.fill_between(monthly_revenue.index, monthly_revenue.values, alpha=0.3, color='darkgreen')
ax3.set_title('Monthly Revenue Trend', fontsize=12, fontweight='bold')
ax3.set_xlabel('Month', fontsize=10)
ax3.set_ylabel('Revenue ($)', fontsize=10)
ax3.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax3.tick_params(axis='x', rotation=45, labelsize=8)
ax3.grid(True, alpha=0.3)

# 4. Monthly ROAS
ax4 = fig.add_subplot(gs[1, 2])
monthly_roas = df.groupby('Month')['ROAS'].mean()
ax4.plot(monthly_roas.index, monthly_roas.values, marker='^', 
         linewidth=2, markersize=6, color='coral')
ax4.axhline(y=monthly_roas.mean(), color='red', linestyle='--', 
            label=f'Avg: {monthly_roas.mean():.2f}x', alpha=0.7)
ax4.set_title('Monthly ROAS', fontsize=12, fontweight='bold')
ax4.set_xlabel('Month', fontsize=10)
ax4.set_ylabel('ROAS (x)', fontsize=10)
ax4.tick_params(axis='x', rotation=45, labelsize=8)
ax4.legend()
ax4.grid(True, alpha=0.3)

# 5. Top Mediums by ROAS
ax5 = fig.add_subplot(gs[2, 0])
top_roas = by_medium.nlargest(6, 'ROAS')[['ROAS']]
colors_roas = ['green' if x > 5 else 'orange' if x > 2 else 'red' for x in top_roas['ROAS']]
ax5.barh(top_roas.index, top_roas['ROAS'], color=colors_roas, alpha=0.7)
ax5.set_title('Top 6 Mediums by ROAS', fontsize=12, fontweight='bold')
ax5.set_xlabel('ROAS (x)', fontsize=10)
ax5.grid(True, alpha=0.3, axis='x')

# 6. Conversion Funnel
ax6 = fig.add_subplot(gs[2, 1])
funnel_data = [
    ('Impressions', df['Impressions'].sum()),
    ('Clicks', df['Clicks'].sum()),
    ('Conversions', df['Total Conversions'].sum()),
    ('Orders', df['Total Orders'].sum())
]
funnel_labels = [x[0] for x in funnel_data]
funnel_values = [x[1] for x in funnel_data]
colors_funnel = ['#FF6B6B', '#4ECDC4', '#45B7D1', '#96CEB4']
ax6.barh(funnel_labels, funnel_values, color=colors_funnel, alpha=0.8)
ax6.set_title('Marketing Funnel', fontsize=12, fontweight='bold')
ax6.set_xlabel('Count', fontsize=10)
ax6.set_xscale('log')
for i, v in enumerate(funnel_values):
    ax6.text(v, i, f' {v:,}', va='center', fontsize=9)

# 7. Quarterly Performance
ax7 = fig.add_subplot(gs[2, 2])
quarterly = df.groupby('Quarter').agg({
    'Ads Spend': 'sum',
    'Revenue': 'sum'
})
x_pos = np.arange(len(quarterly))
width = 0.35
ax7.bar(x_pos - width/2, quarterly['Ads Spend'], width, 
        label='Spend', color='lightcoral', alpha=0.8)
ax7.bar(x_pos + width/2, quarterly['Revenue'], width, 
        label='Revenue', color='lightgreen', alpha=0.8)
ax7.set_xticks(x_pos)
ax7.set_xticklabels([f'Q{i+1}' for i in range(len(quarterly))])
ax7.set_title('Quarterly Spend vs Revenue', fontsize=12, fontweight='bold')
ax7.set_ylabel('Amount ($)', fontsize=10)
ax7.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax7.legend()
ax7.grid(True, alpha=0.3, axis='y')

plt.suptitle('Marketing Analytics Executive Dashboard - 2018', 
             fontsize=18, fontweight='bold', y=0.995)
plt.show()

## Channel Performance Deep Dive

In [ ]:
# Channel efficiency metrics
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# CTR by Medium
ctr_sorted = by_medium.sort_values('CTR', ascending=True)
axes[0, 0].barh(ctr_sorted.index, ctr_sorted['CTR'], color='skyblue')
axes[0, 0].set_title('Click-Through Rate by Medium', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('CTR (%)', fontsize=11)
axes[0, 0].grid(True, alpha=0.3, axis='x')

# Conversion Rate by Medium
conv_sorted = by_medium.sort_values('Conversion Rate', ascending=True)
axes[0, 1].barh(conv_sorted.index, conv_sorted['Conversion Rate'], color='lightgreen')
axes[0, 1].set_title('Conversion Rate by Medium', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Conversion Rate (%)', fontsize=11)
axes[0, 1].grid(True, alpha=0.3, axis='x')

# Revenue by Medium
revenue_sorted = by_medium.sort_values('Revenue', ascending=True)
axes[1, 0].barh(revenue_sorted.index, revenue_sorted['Revenue'], color='lightcoral')
axes[1, 0].set_title('Total Revenue by Medium', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Revenue ($)', fontsize=11)
axes[1, 0].xaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
axes[1, 0].grid(True, alpha=0.3, axis='x')

# ROI by Medium
roi_sorted = by_medium.sort_values('ROI', ascending=True)
colors_roi = ['red' if x < 0 else 'orange' if x < 100 else 'green' for x in roi_sorted['ROI']]
axes[1, 1].barh(roi_sorted.index, roi_sorted['ROI'], color=colors_roi, alpha=0.7)
axes[1, 1].axvline(x=0, color='black', linestyle='-', linewidth=1)
axes[1, 1].set_title('ROI by Medium', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('ROI (%)', fontsize=11)
axes[1, 1].xaxis.set_major_formatter(mtick.StrMethodFormatter('{x:.0f}%'))
axes[1, 1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

## Spend vs Revenue Analysis

In [ ]:
# Scatter plot: Spend vs Revenue by Medium
fig, ax = plt.subplots(figsize=(14, 8))

for medium in by_medium.index:
    medium_data = df[df['Medium'] == medium]
    ax.scatter(
        medium_data['Ads Spend'],
        medium_data['Revenue'],
        s=100,
        alpha=0.6,
        label=medium
    )

# Add break-even line
max_val = max(df['Ads Spend'].max(), df['Revenue'].max())
ax.plot([0, max_val], [0, max_val], 'r--', linewidth=2, 
        label='Break-even', alpha=0.5)

ax.set_xlabel('Ad Spend ($)', fontsize=13)
ax.set_ylabel('Revenue ($)', fontsize=13)
ax.set_title('Marketing Spend vs Revenue by Medium', fontsize=16, fontweight='bold')
ax.xaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Monthly Trends Comparison

In [ ]:
# Monthly metrics comparison
monthly_metrics = df.groupby('Month').agg({
    'Ads Spend': 'sum',
    'Revenue': 'sum',
    'Total Orders': 'sum',
    'Total Conversions': 'sum',
    'ROAS': 'mean'
}).round(2)

fig, axes = plt.subplots(3, 1, figsize=(16, 12))

# Spend vs Revenue
ax1_twin = axes[0].twinx()
line1 = axes[0].plot(monthly_metrics.index, monthly_metrics['Ads Spend'], 
                     marker='o', linewidth=2.5, color='#FF6B6B', label='Ad Spend')
line2 = ax1_twin.plot(monthly_metrics.index, monthly_metrics['Revenue'], 
                      marker='s', linewidth=2.5, color='#4ECDC4', label='Revenue')
axes[0].set_xlabel('Month', fontsize=11)
axes[0].set_ylabel('Ad Spend ($)', fontsize=11, color='#FF6B6B')
ax1_twin.set_ylabel('Revenue ($)', fontsize=11, color='#4ECDC4')
axes[0].set_title('Monthly Spend vs Revenue', fontsize=14, fontweight='bold')
axes[0].yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
ax1_twin.yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
axes[0].tick_params(axis='y', labelcolor='#FF6B6B')
ax1_twin.tick_params(axis='y', labelcolor='#4ECDC4')
axes[0].grid(True, alpha=0.3)
lines = line1 + line2
labels = [l.get_label() for l in lines]
axes[0].legend(lines, labels, loc='upper left')

# Orders and Conversions
ax2_twin = axes[1].twinx()
line3 = axes[1].plot(monthly_metrics.index, monthly_metrics['Total Orders'], 
                     marker='o', linewidth=2.5, color='#45B7D1', label='Orders')
line4 = ax2_twin.plot(monthly_metrics.index, monthly_metrics['Total Conversions'], 
                      marker='s', linewidth=2.5, color='#96CEB4', label='Conversions')
axes[1].set_xlabel('Month', fontsize=11)
axes[1].set_ylabel('Total Orders', fontsize=11, color='#45B7D1')
ax2_twin.set_ylabel('Total Conversions', fontsize=11, color='#96CEB4')
axes[1].set_title('Monthly Orders and Conversions', fontsize=14, fontweight='bold')
axes[1].tick_params(axis='y', labelcolor='#45B7D1')
ax2_twin.tick_params(axis='y', labelcolor='#96CEB4')
axes[1].grid(True, alpha=0.3)
lines = line3 + line4
labels = [l.get_label() for l in lines]
axes[1].legend(lines, labels, loc='upper left')

# ROAS Trend
axes[2].plot(monthly_metrics.index, monthly_metrics['ROAS'], 
             marker='D', linewidth=3, markersize=8, color='#D4A5A5')
axes[2].axhline(y=monthly_metrics['ROAS'].mean(), color='red', 
                linestyle='--', linewidth=2, 
                label=f'Average: {monthly_metrics["ROAS"].mean():.2f}x')
axes[2].fill_between(monthly_metrics.index, monthly_metrics['ROAS'], 
                      alpha=0.3, color='#D4A5A5')
axes[2].set_xlabel('Month', fontsize=11)
axes[2].set_ylabel('ROAS (x)', fontsize=11)
axes[2].set_title('Monthly Return on Ad Spend (ROAS)', fontsize=14, fontweight='bold')
axes[2].legend()
axes[2].grid(True, alpha=0.3)

for ax in axes:
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

## Budget Efficiency Analysis

In [ ]:
# Calculate budget variance
df['Budget Variance'] = df['Marketing Invoice'] - df['Marketing Estimate']
df['Budget Variance %'] = (df['Budget Variance'] / df['Marketing Estimate'] * 100).round(2)

variance_by_medium = df.groupby('Medium').agg({
    'Marketing Estimate': 'sum',
    'Marketing Invoice': 'sum',
    'Budget Variance': 'sum'
}).round(2)

variance_by_medium['Variance %'] = (
    variance_by_medium['Budget Variance'] / variance_by_medium['Marketing Estimate'] * 100
).round(2)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Budget vs Actual by Medium
x = np.arange(len(variance_by_medium))
width = 0.35
bars1 = axes[0].bar(x - width/2, variance_by_medium['Marketing Estimate'], 
                    width, label='Budget', color='lightblue', alpha=0.8)
bars2 = axes[0].bar(x + width/2, variance_by_medium['Marketing Invoice'], 
                    width, label='Actual', color='coral', alpha=0.8)
axes[0].set_xticks(x)
axes[0].set_xticklabels(variance_by_medium.index, rotation=45, ha='right')
axes[0].set_ylabel('Amount ($)', fontsize=12)
axes[0].set_title('Budget vs Actual Spend by Medium', fontsize=14, fontweight='bold')
axes[0].yaxis.set_major_formatter(mtick.StrMethodFormatter('${x:,.0f}'))
axes[0].legend()
axes[0].grid(True, alpha=0.3, axis='y')

# Variance %
variance_sorted = variance_by_medium.sort_values('Variance %')
colors = ['green' if x < 0 else 'red' for x in variance_sorted['Variance %']]
axes[1].barh(variance_sorted.index, variance_sorted['Variance %'], 
             color=colors, alpha=0.7)
axes[1].axvline(x=0, color='black', linestyle='-', linewidth=1)
axes[1].set_xlabel('Variance (%)', fontsize=12)
axes[1].set_title('Budget Variance % by Medium', fontsize=14, fontweight='bold')
axes[1].xaxis.set_major_formatter(mtick.StrMethodFormatter('{x:.1f}%'))
axes[1].grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\nGreen = Under Budget, Red = Over Budget")

## Conversion Analysis

In [ ]:
# Conversion type breakdown
conversion_types = df.groupby('Medium').agg({
    'Add To Cart': 'sum',
    'Lead Submission': 'sum',
    'Whitepaper Download': 'sum'
})

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Stacked bar chart
conversion_types.plot(kind='bar', stacked=True, ax=axes[0],
                      color=['#FF6B6B', '#4ECDC4', '#45B7D1'])
axes[0].set_title('Conversions by Type and Medium', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Medium', fontsize=12)
axes[0].set_ylabel('Number of Conversions', fontsize=12)
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')
axes[0].legend(title='Conversion Type', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].grid(True, alpha=0.3, axis='y')

# Conversion rate by medium
conv_rate_sorted = by_medium.sort_values('Conversion Rate', ascending=False)
colors_conv = sns.color_palette('RdYlGn', len(conv_rate_sorted))
axes[1].bar(range(len(conv_rate_sorted)), conv_rate_sorted['Conversion Rate'], 
            color=colors_conv, alpha=0.8)
axes[1].set_xticks(range(len(conv_rate_sorted)))
axes[1].set_xticklabels(conv_rate_sorted.index, rotation=45, ha='right')
axes[1].set_ylabel('Conversion Rate (%)', fontsize=12)
axes[1].set_title('Conversion Rate by Medium (Ranked)', fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Top Performers Summary

In [ ]:
print("="*80)
print("TOP PERFORMING MARKETING CHANNELS - 2018")
print("="*80)

print("\n📊 Top 3 by Revenue:")
top_revenue = by_medium.nlargest(3, 'Revenue')[['Revenue', 'Ads Spend', 'ROAS']]
for idx, row in top_revenue.iterrows():
    print(f"  {idx}: ${row['Revenue']:,.2f} (Spend: ${row['Ads Spend']:,.2f}, ROAS: {row['ROAS']:.2f}x)")

print("\n💰 Top 3 by ROAS:")
top_roas = by_medium.nlargest(3, 'ROAS')[['ROAS', 'Revenue', 'Ads Spend']]
for idx, row in top_roas.iterrows():
    print(f"  {idx}: {row['ROAS']:.2f}x (Revenue: ${row['Revenue']:,.2f}, Spend: ${row['Ads Spend']:,.2f})")

print("\n🎯 Top 3 by Conversion Rate:")
top_conv = by_medium.nlargest(3, 'Conversion Rate')[['Conversion Rate', 'Total Conversions']]
for idx, row in top_conv.iterrows():
    print(f"  {idx}: {row['Conversion Rate']:.2f}% ({int(row['Total Conversions']):,} conversions)")

print("\n📈 Top 3 by ROI:")
top_roi = by_medium.nlargest(3, 'ROI')[['ROI', 'Revenue', 'Ads Spend']]
for idx, row in top_roi.iterrows():
    print(f"  {idx}: {row['ROI']:.2f}% (Revenue: ${row['Revenue']:,.2f}, Spend: ${row['Ads Spend']:,.2f})")

print("\n" + "="*80)

## Save Processed Data

In [ ]:
# Save combined dataset
df.to_csv('marketing_analytics_combined.csv', index=False)
by_medium.to_csv('performance_by_medium.csv')
monthly_metrics.to_csv('monthly_metrics.csv')

print("Data saved successfully!")
print(f"- marketing_analytics_combined.csv ({len(df)} records)")
print(f"- performance_by_medium.csv ({len(by_medium)} mediums)")
print(f"- monthly_metrics.csv ({len(monthly_metrics)} months)")